In [1]:
import glob
import json
import os
import csv
import zstandard as zstd
import io

KEYWORDS = [
    "election",
    "Obama",
    "McCain",
    "campaign",
    "vote",
    "crisis",
    "recession",
    "bailout",
    "bankruptcy",
    "subprime",
    "housing bubble"
]

SELECTED_COLUMNS = [
    "id", "author", "created_utc", "title", "selftext", "score",
    "num_comments", "subreddit", "permalink", "author_flair_text"
]

In [2]:
def is_relevant(obj, keywords=KEYWORDS):
    text = ""
    for field in ["title", "selftext", "body"]:
        if field in obj and obj[field]:
            text += " " + str(obj[field])
    text = text.lower()
    return any(keyword.lower() in text for keyword in keywords)

def process_zst_files(input_folder, output_csv_zst):
    zst_files = glob.glob(os.path.join(input_folder, '*.zst'))
    if not zst_files:
        print("No .zst files found in the folder.")
        return

    temp_csv = output_csv_zst + '.tmp'
    
    with open(temp_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=SELECTED_COLUMNS, extrasaction='ignore')
        writer.writeheader()  # Write header once

        for zst_file in zst_files:
            print(f"Processing file: {zst_file}")
            with open(zst_file, 'rb') as f:
                dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
                with dctx.stream_reader(f) as reader:
                    text_stream = io.TextIOWrapper(reader, encoding='utf-8')
                    for line in text_stream:
                        try:
                            obj = json.loads(line)
                        except Exception as e:
                            print(f"Error decoding line in {zst_file}: {e}")
                            continue
                        
                        if not is_relevant(obj):
                            continue

                        filtered_obj = {col: obj.get(col, None) for col in SELECTED_COLUMNS}
                        writer.writerow(filtered_obj)

    with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
        cctx = zstd.ZstdCompressor()
        cctx.copy_stream(input_f, output_f)

    os.remove(temp_csv)
    print(f"Compressed CSV written to {output_csv_zst}")

In [4]:
input_folder = 'submissions'
output_csv_zst = 'submissions.csv.zst'
process_zst_files(input_folder, output_csv_zst)

Processing file: submissions/RS_2008-10.zst
Processing file: submissions/RS_2008-11.zst
Processing file: submissions/RS_2008-12.zst
Compressed CSV written to submissions.csv.zst


In [5]:
import pandas as pd
import zstandard as zstd
import io

def read_csv_zst(file_path):
    with open(file_path, 'rb') as compressed_file:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(compressed_file) as reader:
            return pd.read_csv(io.BytesIO(reader.read()))

# Usage
df = read_csv_zst("submissions.csv.zst")

In [6]:
df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,74i2t,[deleted],1222819433,John McCain was not only fully aware of the Bu...,[deleted],1,0,politics,/r/politics/comments/74i2t/john_mccain_was_not...,NaN
1,74i2x,Anderkay,1222819466,What economic crisis? Check out Dubai's $1.5Bi...,NaN,2,0,reddit.com,/r/reddit.com/comments/74i2x/what_economic_cri...,NaN
2,74i30,IAmperfectlyCalm,1222819518,John McCain was not only fully aware of the Bu...,NaN,75,14,politics,/r/politics/comments/74i30/john_mccain_was_not...,NaN
3,74i3i,[deleted],1222819718,McCain rumoured to suspend his campaign once a...,NaN,3,2,politics,/r/politics/comments/74i3i/mccain_rumoured_to_...,NaN
4,74i3j,georgewashingtonblog,1222819723,Republicans Released Ad Attacking Dems For PAS...,NaN,11,2,politics,/r/politics/comments/74i3j/republicans_release...,NaN


In [7]:
import pandas as pd

subreddits = [
    "politics",
    "news",
    "worldnews",
    "Ask_Politics",
    "conspiracy",
    "Conservative",
    "liberal"
]

In [8]:
filtered_df = df[df["subreddit"].isin(subreddits)]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
0,74i2t,[deleted],1222819433,John McCain was not only fully aware of the Bu...,[deleted],1,0,politics,/r/politics/comments/74i2t/john_mccain_was_not...,NaN
2,74i30,IAmperfectlyCalm,1222819518,John McCain was not only fully aware of the Bu...,NaN,75,14,politics,/r/politics/comments/74i30/john_mccain_was_not...,NaN
3,74i3i,[deleted],1222819718,McCain rumoured to suspend his campaign once a...,NaN,3,2,politics,/r/politics/comments/74i3i/mccain_rumoured_to_...,NaN
4,74i3j,georgewashingtonblog,1222819723,Republicans Released Ad Attacking Dems For PAS...,NaN,11,2,politics,/r/politics/comments/74i3j/republicans_release...,NaN
7,74i42,mpinkeyes,1222819884,Investigation into the “Stolen Vote” Proves Vo...,NaN,3,0,politics,/r/politics/comments/74i42/investigation_into_...,NaN


In [9]:
filtered_df = filtered_df[filtered_df["author"] != "[deleted]"]

filtered_df.head()

,id,author,created_utc,title,selftext,score,num_comments,subreddit,permalink,author_flair_text
2,74i30,IAmperfectlyCalm,1222819518,John McCain was not only fully aware of the Bu...,NaN,75,14,politics,/r/politics/comments/74i30/john_mccain_was_not...,NaN
4,74i3j,georgewashingtonblog,1222819723,Republicans Released Ad Attacking Dems For PAS...,NaN,11,2,politics,/r/politics/comments/74i3j/republicans_release...,NaN
7,74i42,mpinkeyes,1222819884,Investigation into the “Stolen Vote” Proves Vo...,NaN,3,0,politics,/r/politics/comments/74i42/investigation_into_...,NaN
10,74i45,georgewashingtonblog,1222819896,"Nobel laureate Stiglitz predicts Obama win, re...",NaN,2,0,politics,/r/politics/comments/74i45/nobel_laureate_stig...,NaN
12,74i48,monkeymynd,1222819918,"Senate to vote on ""financial rescue plan"" Wedn...",NaN,22,5,politics,/r/politics/comments/74i48/senate_to_vote_on_f...,NaN


In [10]:
import pandas as pd
import zstandard as zstd
import os

output_csv_zst = "submissions.csv.zst"

temp_csv = "filtered_subreddits.csv"
filtered_df.to_csv(temp_csv, index=False, encoding="utf-8")

with open(temp_csv, 'rb') as input_f, open(output_csv_zst, 'wb') as output_f:
    cctx = zstd.ZstdCompressor()
    cctx.copy_stream(input_f, output_f)

# Remove the temporary CSV file
os.remove(temp_csv)

print(f"Compressed CSV saved as {output_csv_zst}")

Compressed CSV saved as submissions.csv.zst
